In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import imageio.v2 as imageio
import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import lightning.pytorch as pl

# Data

## Read data

In [2]:
data_dir = r'F:\DGM\code\HW2\Data\small'

image_list = []
label_list = []

for root,dirs,files in os.walk(data_dir):

    if len(files) != 0:
        label = root.split('\\')[-1]
        print(f'Reading folder {label}')
        for i in tqdm.trange(len(files)):
            file = files[i]
            file_path = os.path.join(root, file)
            image = imageio.imread(file_path)
            image_list.append(image)
            label_list.append(label)

im_size = image.shape

image_array = np.array(image_list)
label_array = np.array(label_list, dtype=np.int32)

Reading folder 0


100%|██████████| 1208/1208 [00:01<00:00, 651.62it/s]


Reading folder 1


100%|██████████| 468/468 [00:00<00:00, 586.07it/s]


Reading folder 2


...

100%|██████████| 1790/1790 [00:01<00:00, 985.18it/s] 


Reading folder 8


100%|██████████| 1280/1280 [00:01<00:00, 954.76it/s]


Reading folder 9


100%|██████████| 1293/1293 [00:01<00:00, 780.00it/s]


## Data preprocessing

In [3]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [4]:
image_train, image_test, label_train, label_test = train_test_split(
    image_array,
    label_array, 
    test_size = 0.2,
    shuffle = True,
    random_state = 69
)

In [5]:
scaler = MinMaxScaler()

image_train_scaled = scaler.fit_transform(image_train.reshape(image_train.shape[0], -1))
image_test_scaled = scaler.transform(image_test.reshape(image_test.shape[0], -1))

image_train_scaled = image_train_scaled.reshape(-1, im_size[0], im_size[1])
image_test_scaled = image_test_scaled.reshape(-1, im_size[0], im_size[1])

In [6]:
def array_2_loader(X, y, bs=32):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    X_tensor = X_tensor.unsqueeze(1)
    y_tensor = torch.tensor(y, dtype=torch.long)
    dataset = TensorDataset(X_tensor, y_tensor)
    dataloader = DataLoader(dataset, batch_size=bs)
    return dataloader

train_loader = array_2_loader(
    image_train_scaled,
    label_train,
    128
)
test_loader = array_2_loader(
    image_test_scaled,
    label_test,
    128
)

# Model

In [7]:
from src.image_gpt import ImageGPT #type: ignore

In [8]:
model = ImageGPT(
    centroids = r'F:\DGM\code\HW3\data\mnist_centroids.npy',
    embed_dim = 16,
    num_heads = 2,
    num_layers = 8,
    num_pixels = im_size[0],
    num_vocab = 16,
    num_classes = 10,
    classify = False,
    steps = 100,
    warmup_steps =10,
)

In [9]:
from lightning.pytorch.callbacks import EarlyStopping
from lightning.pytorch.callbacks import Callback
from lightning.pytorch.loggers import CSVLogger


In [10]:
early_stop_callback = EarlyStopping(
    monitor="val_loss",  # Track validation loss
    patience=3,          # Number of epochs to wait before stopping
    mode="min"           # "min" since we want the lowest loss
)

csv_logger = CSVLogger(save_dir="logs", name="my_model")


class LossTrackerCallback(Callback):
    def __init__(self):
        self.loss_dict = {"train_loss": [], "val_loss": []}

    def on_epoch_end(self, trainer, pl_module):
        # Retrieve metrics at the end of each epoch
        metrics = trainer.callback_metrics
        if "train_loss" in metrics:
            self.loss_dict["train_loss"].append(metrics["train_loss"].item())
        if "val_loss" in metrics:
            self.loss_dict["val_loss"].append(metrics["val_loss"].item())

    def get_loss_dict(self):
        """Returns the stored loss dictionary"""
        return self.loss_dict


In [11]:
trainer = pl.Trainer(
    accelerator = 'auto',
    max_epochs = 10,
    # limit_train_batches = 3,
    limit_val_batches = 3,
    callbacks = [early_stop_callback, LossTrackerCallback()],
    logger = [csv_logger]
)

trainer.fit(model, train_loader, test_loader)

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type             | Params | Mode 
----------------------------------------------------------
0 | gpt          | GPT2             | 33.4 K | train
1 | criterion    | CrossEntropyLoss | 0      | train
  | other params | n/a              | 16     | n/a  
----------------------------------------------------------
33.4 K    Trainable params
16        Non-trainable params
33.4 K    Total params
0.134     Total estimated model params size (MB)
80        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

g:\Anaconda3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
g:\Anaconda3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined